<a href="https://colab.research.google.com/github/jayanth1310/AI-Based-Interior-Design/blob/main/AI_Baesd_Interior_Design(latest).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AI Based Interior Design Code

In [ ]:
!pip install streamlit pyngrok diffusers transformers accelerate opencv-python pillow -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 112.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 105.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 116.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 88.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21

In [ ]:

from pyngrok import ngrok
import streamlit as st
import torch
from PIL import Image
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel, UniPCMultistepScheduler
from transformers import DPTFeatureExtractor, DPTForDepthEstimation
import numpy as np
import cv2


In [ ]:
%%writefile app.py
import streamlit as st
import torch
from PIL import Image
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel, UniPCMultistepScheduler
from transformers import DPTImageProcessor, DPTForDepthEstimation
import numpy as np
import cv2
import os

# Fix for CUDA/cuDNN warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # Suppress TensorFlow logs
torch.backends.cudnn.benchmark = True  # Optimize CUDA

# Fix for Streamlit event loop error
import asyncio
if not hasattr(asyncio, '_nest_patched'):
    asyncio._nest_patched = True  # Patch for Colab's async environment

def preprocess_image(image):
    """Preprocess image to match required dimensions."""
    original_width, original_height = image.size
    adjusted_width = adjust_dimensions(original_width)
    adjusted_height = adjust_dimensions(original_height)
    return image.resize((adjusted_width, adjusted_height))

def adjust_dimensions(size):
    return size - (size % 8) if (size % 8) != 0 else size

def process_and_generate(image, prompt):
    """Process the image and generate the new design."""
    resized_image = preprocess_image(image)
    depth_map = generate_depth_map(resized_image)
    return generate_design(depth_map, prompt, resized_image.size)

#streamlit app creation
def streamlit_generation():
  st.title("AI Interior Designer 🛋️")
  uploaded_image = st.file_uploader("Upload a room photo", type=["jpg", "png"])
  prompt = st.text_input("Design prompt:")
  with st.spinner("Generating your design..."):
        output_image = pipe(
            prompt=prompt,
            image=depth_map,
            height=adjusted_height,
            width=adjusted_width,
            num_inference_steps=30,
            guidance_scale=7.5,
            controlnet_conditioning_scale=0.8
        ).images[0]

# Load models
@st.cache_resource
def load_models():
    depth_estimator = DPTForDepthEstimation.from_pretrained("Intel/dpt-large")
    image_processor = DPTImageProcessor.from_pretrained("Intel/dpt-large")

    controlnet = ControlNetModel.from_pretrained(
        "lllyasviel/sd-controlnet-depth",
        torch_dtype=torch.float16
    )
    pipe = StableDiffusionControlNetPipeline.from_pretrained(
        "runwayml/stable-diffusion-v1-5",
        controlnet=controlnet,
        torch_dtype=torch.float16
    ).to("cuda")
    pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)
    return pipe, depth_estimator, image_processor

def generate_depth_map(image):
    inputs = image_processor(images=image, return_tensors="pt")
    with torch.no_grad():
        depth_output = depth_estimator(**inputs).predicted_depth
    depth = depth_output.numpy()
    depth = (depth - depth.min()) / (depth.max() - depth.min()) * 255.0
    depth = depth.astype(np.uint8)
    return Image.fromarray(depth.squeeze())

def is_valid_prompt(prompt):
    """Checks if the prompt is related to room design changes."""
    allowed_keywords = [
        "sofa", "table", "chair", "bed", "wall", "carpet", "curtains", "room", "furniture", "interior", "decor",
        "lamp", "bookshelf", "painting", "ceiling", "floor", "lighting", "couch", "vase", "mirror", "window",
        "blinds", "cushion", "coffee table", "side table", "wardrobe", "drapes", "wall art", "shelves", "desk",
        "television", "cabinet", "ottoman", "chandelier", "tiles", "color scheme", "plants", "accent wall",
        "bedside table", "rug", "frame", "pillows", "headboard", "nightstand", "workspace", "storage",
        "fireplace", "doors", "bedding", "paneling", "molding", "baseboards", "wallpaper", "built-in shelves"
    ]
    return any(keyword in prompt.lower() for keyword in allowed_keywords)

# Streamlit UI
st.title("AI Interior Designer 🛋️")
uploaded_image = st.file_uploader("Upload a room photo", type=["jpg", "png"])
prompt = st.text_input("Design prompt:")

if uploaded_image and prompt:
    if not is_valid_prompt(prompt):
        st.error("Invalid prompt! Please enter a design prompt related to room changes (e.g., sofa, wall, furniture, decor).")
        st.stop()

    pipe, depth_estimator, image_processor = load_models()

    # Process image and get dimensions
    input_image = Image.open(uploaded_image).convert("RGB")
    original_width, original_height = input_image.size

    # Adjust dimensions to multiples of 8 (required by Stable Diffusion)
    def adjust_dimensions(size):
        return size - (size % 8) if (size % 8) != 0 else size

    adjusted_width = adjust_dimensions(original_width)
    adjusted_height = adjust_dimensions(original_height)

    # Resize input image to adjusted dimensions (optional but recommended)
    input_image = input_image.resize((adjusted_width, adjusted_height))

    # Generate depth map
    depth_map = generate_depth_map(input_image)

    # Generate output image with explicit dimensions
    with st.spinner("Generating your design..."):
        output_image = pipe(
            prompt=prompt,
            image=depth_map,
            height=adjusted_height,
            width=adjusted_width,
            num_inference_steps=40,
            guidance_scale=7.5,
            controlnet_conditioning_scale=0.8
        ).images[0]

    # Resize output back to original dimensions (optional)
    output_image = output_image.resize((original_width, original_height))

    # Display results
    st.subheader("Original Design")
    st.image(input_image, use_container_width=True)

    st.subheader("New Design")
    st.image(output_image, use_container_width=True)

Overwriting app.py


In [ ]:
ngrok.set_auth_token("2ui3YOhepTXwGzNHw1MMXsxYgYd_23SXGg8gwPKyR6TVWh9Ji")
public_url = ngrok.connect(addr="8501", proto="http", bind_tls=True)
print("Streamlit URL:", public_url)

# Run Streamlit in the background
!streamlit run app.py --server.port 8501 --browser.serverAddress 0.0.0.0 --server.enableCORS False

Streamlit URL: NgrokTunnel: "https://03ac-34-126-147-220.ngrok-free.app" -> "http://localhost:8501"
2025-03-24 17:21:07.111 
As a result, 'server.enableCORS' is being overridden to 'true'.

More information:
In order to protect against CSRF attacks, we send a cookie with each request.
To do so, we must specify allowable origins, which places a restriction on
cross-origin resource sharing.

If cross origin resource sharing is required, please disable server.enableXsrfProtection.
            



  You can now view your Streamlit app in your browser.

  URL: http://0.0.0.0:8501

2025-03-24 17:21:24.571639: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742836884.595497    5620 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742836884.603233   